<a href="https://colab.research.google.com/github/Muun-Muun/clusterd_sattelites_optimization/blob/main/CWPlotter_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
from matplotlib.colors import LogNorm
from matplotlib.lines import Line2D
from numpy.linalg import inv, eigh, norm
import numpy as np


class CWPlotter:
    def __init__(self):
        pass

    def draw_pc_contour(self, r_rel, C_rel, ax, range=(10, 10), HBR=1.0, density=10, alpha=0.5):
        C_inv = inv(C_rel)
        eigvals, eigvecs = eigh(C_rel)
        angle = np.degrees(np.arctan2(*eigvecs[:, 0][::-1]))
        width, height = 2 * np.sqrt(eigvals)
        sigma_x, sigma_y = np.sqrt(C_rel[0, 0]), np.sqrt(C_rel[1, 1])

        x_vals = np.linspace(-range[0], range[0], 400)
        z_vals = np.linspace(-range[1], range[1], 400)
        X, Z = np.meshgrid(x_vals, z_vals)

        d2_grid = np.einsum('...i,ij,...j->...', np.dstack((X, Z)), C_inv, np.dstack((X, Z)))
        pre_factor = HBR ** 2 / (2 * sigma_x * sigma_y)
        Pc = pre_factor * np.exp(-0.5 * d2_grid)

        levels = np.logspace(-10, -0, num=density)
        contour = ax.contour(X, Z, Pc, levels=levels, cmap='rainbow', norm=LogNorm(), alpha=alpha)
        ax.clabel(contour, inline=True, fmt=lambda v: f"{v:.0e}", fontsize=8)

        ellipse = Ellipse((0, 0), width=width, height=height, angle=angle,
                          edgecolor='blue', linestyle='--', lw=1.5,
                          facecolor='none', label='Covariance Ellipse (1σ)')
        ax.add_patch(ellipse)

    def plot_covariance_ellipse(self, cov, ax, n_std=1.0, edgecolor='blue', **kwargs):
        eigvals, eigvecs = np.linalg.eigh(cov)
        order = eigvals.argsort()[::-1]
        eigvals, eigvecs = eigvals[order], eigvecs[:, order]
        width, height = 2 * n_std * np.sqrt(eigvals)
        angle = np.degrees(np.arctan2(*eigvecs[:, 0][::-1]))
        ellipse = Ellipse(xy=(0, 0), width=width, height=height, angle=angle,
                          edgecolor=edgecolor, facecolor='none', **kwargs)
        ax.add_patch(ellipse)

    def plot_dv_arrow(self, ax, position, delta_v, scale=1.0, color='black', label=None):
        x0, y0 = position[0], position[1]
        dx, dy = delta_v[0] * scale, delta_v[1] * scale
        ax.quiver(x0, y0, dx, dy, angles='xy', scale_units='xy', scale=1,
                  color=color, width=0.005)

        if label:
            legend_handle = Line2D([0], [0], color=color, lw=2, marker='>',
                                   markersize=8, label=label)
            return legend_handle
        else:
            return None
